# 최종 크롤링

In [16]:
import selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
import re

In [17]:
selenium.__version__

'4.12.0'

In [20]:
#지영의 함수 - 최종

from selenium import webdriver
from selenium.webdriver.common.by import By
import re

def cnbc_crawling(start_date, end_date):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')

    start_values_iter = iter(range(0, 320, 10))
    all_matching_links = []
    retry_count = 0
    terminate_flag = False

    while True:
        if terminate_flag:
            break

        try:
            current_start = next(start_values_iter)
        except StopIteration:
            break

        print(f"page: {current_start // 10 + 1}")

        while True:
            driver = webdriver.Chrome(options=chrome_options)
            matching_links_count = 0
            date_range_str = f"cd_min:{start_date},cd_max:{end_date}"
            google_search_url = f'https://www.google.com/search?q=cnbc&sca_esv=568517199&tbs=cdr:1,{date_range_str}&tbm=nws&sxsrf=AM9HkKmYdZ0_zRMPireMpaE6VsIgqW5jBg:1695740222154&ei=PvESZZmECbfh2roPnbOa-A0&start={current_start}&sa=N&ved=2ahUKEwiZm8rMxMiBAxW3sFYBHZ2ZBt84ygIQ8tMDegQIAxAW&biw=1137&bih=790&dpr=2'
            driver.get(google_search_url)

            links_with_class = driver.find_elements(By.CSS_SELECTOR, 'a.WlydOe[jsname="YKoRaf"]')
            for link in links_with_class:
                href = link.get_attribute('href')
                if href and re.match(r'https://www\.cnbc\.com/2023/\d{2}/\d{2}/[a-z0-9-]+\.html', href):
                    all_matching_links.append(href)
                    print(href)
                    matching_links_count += 1

            if matching_links_count > 0:
                retry_count = 0
                break
            else:
                driver.quit()
                retry_count += 1

            if retry_count >= 2:
                terminate_flag = True
                break
    
    return all_matching_links


#함수 호출
#cnbc_crawling("08/24/2023", "08/31/2023")

In [21]:
#저장코드 추가

from datetime import datetime, timedelta
import pickle  # make sure to import pickle

# Initialize variables
start_date = datetime.strptime("7/16/2023", "%m/%d/%Y")
end_date = datetime.strptime("8/31/2023", "%m/%d/%Y")
all_matching_links = []
counter = 0  # Initialize counter variable

# Generate date ranges
while start_date <= end_date:
    next_end_date = start_date + timedelta(days=6)
    if next_end_date > end_date:
        next_end_date = end_date
    
    # Convert to string format
    current_start_date, current_end_date = (start_date.strftime("%m/%d/%Y"), next_end_date.strftime("%m/%d/%Y"))
    current_matching_links = cnbc_crawling(current_start_date, current_end_date)
    all_matching_links += current_matching_links

    # 결과 처리 또는 저장
    print(f'크롤링 결과 (시작 날짜: {current_start_date}, 종료 날짜: {current_end_date}):')
    print(f'누적 크롤링된 기사 수: {len(all_matching_links)}')
    print(f'현재 크롤링된 기사 수: {len(current_matching_links)}')
    
    start_date += timedelta(days=7)
    
    counter += 1  # Increment counter
    if counter == 4:  # Check if 4 weeks have passed
        with open(f'C:\\Users\\Administrator\\Bitamin_11\\dacon_nh\\dacon_venv\\dataset\\all_matching_links_{next_end_date.strftime("%Y_%m_%d")}.pkl', 'wb') as f:
            pickle.dump(all_matching_links, f)
        counter = 0  # Reset counter

    # Save the remaining data if next_end_date reaches end_date
if next_end_date == end_date:
    with open(f'C:\\Users\\Administrator\\Bitamin_11\\dacon_nh\\dacon_venv\\dataset\\all_matching_links_{next_end_date.strftime("%Y_%m_%d")}.pkl', 'wb') as f:
        pickle.dump(all_matching_links, f)


page: 1
https://www.cnbc.com/2023/07/18/google-restricting-internet-access-to-some-employees-for-security.html
https://www.cnbc.com/2023/07/21/how-bmw-uses-ai-to-make-vehicle-assembly-more-efficient.html
https://www.cnbc.com/2023/07/21/espn-had-talks-with-nba-nfl-in-search-for-strategic-partner.html
https://www.cnbc.com/2023/07/18/frozen-food-grocery-sales-vs-critical-cold-storage-supply-chains-.html
https://www.cnbc.com/2023/07/18/france-and-britain-are-battling-it-out-for-europes-ai-crown.html
https://www.cnbc.com/2023/07/19/millennials-dog-sitting-side-hustle-its-like-a-second-salary.html
https://www.cnbc.com/2023/07/21/large-majority-of-americans-have-strong-reservations-about-ai-emergence-cnbc-survey-shows.html
https://www.cnbc.com/2023/07/19/the-number-one-red-flag-on-a-resume-according-to-a-former-google-recruiter.html
https://www.cnbc.com/2023/07/22/oil-markets-to-face-serious-problems-as-demand-rises-ief.html
page: 2
https://www.cnbc.com/2023/07/20/sadek-wahba-i-squared-capita

### 저장 코드(이상하니까 안 봐도 됨)

In [5]:
import pandas as pd
import pickle

pkl_all = pd.read_pickle("C:\\Users\\Administrator\\Bitamin_11\\dacon_nh\\dacon_venv\\dataset\\all_matching_links_final.pkl")

In [6]:
print(len(pkl_all))
print(len(set(pkl_all)))

5634
5634


In [3]:
pkl_all = set(pkl_all)
pkl_all

{'https://www.cnbc.com/2023/01/06/russia-ukraine-live-updates.html',
 'https://www.cnbc.com/2023/07/21/espn-had-talks-with-nba-nfl-in-search-for-strategic-partner.html',
 'https://www.cnbc.com/2023/08/31/biden-says-he-will-visit-florida-this-weekend-to-survey-idalia-damage.html',
 'https://www.cnbc.com/2023/05/02/ford-cutting-price-of-electric-mustang-mach-e-by-thousands-of-dollars.html',
 'https://www.cnbc.com/2023/04/20/eu-lawmakers-approve-worlds-first-comprehensive-crypto-regulation.html',
 'https://www.cnbc.com/2023/05/02/one-pilot-planes-are-a-possibility-as-aviation-embraces-ai-emirates-president-says.html',
 'https://www.cnbc.com/2023/06/23/ai-has-a-discrimination-problem-in-banking-that-can-be-devastating.html',
 'https://www.cnbc.com/2023/05/18/disney-worlds-star-wars-galactic-starcruiser-to-shutter.html',
 'https://www.cnbc.com/2023/08/24/south-koreas-naver-launches-generative-ai-services-to-compete-with-chatgpt.html',
 'https://www.cnbc.com/2023/06/16/converting-gas-powered

In [4]:
with open(f'C:\\Users\\Administrator\\Bitamin_11\\dacon_nh\\dacon_venv\\dataset\\all_matching_links_final.pkl', 'wb') as f:
        pickle.dump(pkl_all, f)